## Overfitting과 Underfitting
1. 과한 학습을 시켜 train 데이터와 거의 동일하게 학습을 시킨 경우, 해당 데이터를 넘는 test data가 속성으로 들어왔을 때 예측 정확도가 오히려 떨어질 수 있음 이를 Overfitting(과적합)이라고 한다

2. 반대로 학습을 너무 적게하는 것 또한 예측력이 떨어져 Underfitting이 일어날 수 있다

이를 해결하는 방법으로는 데이터(피처)의 양을 조절, 학습의 양을 조절, 알고리즘의 하이퍼파라미터 조정 등이 있다
이 중 굳이 테스트 단계로 넘어가지 않아도, 학습 단계에서 '검증' 하는 방법이 있는데 이것이 중첩이다


### 중첩(fold) - 테스트 데이터에 가기 전 과적합을 피하는 방법
이 중 train data의 학습과 더불어서 Validation(검증) 
즉, 훈련에 대한 검증을 통해 과적합을 방지할 수 있다

검증을 위한 검증 데이터 셋은 '중첩'하여 만든다
1) 학습데이터를 k개의 조각으로 나눔
2) k개로 조각된 train data의 1개를 검증데이터로 사용, 나머지 (K-1) 개를 학습데이터로 사용
3) 따라서 각각의 조각들을 한번씩 검증데이터로 사용하여 교차검증을 진행하는 것
4) 이 k개의 조각을 fold라 하며 k차 검증 즉, K fold(중첩)을 통해 훈련과정에서 과적합을 피할 수 있음
5) '검증'은 테스트 단계가 아닌 train 단계에서 일어나는 것
6) --> K fold cross validation

## sklearn.model_selection.KFold 와 KFold의 유형
KFold(n_splits=5, *, shuffle=False, random_state=None)
split 하이퍼 파라미터 값에 따라서 몇 개의 중첩검증을 진행할 것인가 
### Normal K fold

### Stratified K fold
1) 불균형한 분포도를 가진 데이터 집합을 균등추출

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(max_depth=2, random_state=12)

#5개의 폴드 세트로 분리하는 kFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits = 5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기: ',features.shape[0])

붓꽃 데이터 세트 크기:  150


In [15]:
n_iter = 0
# KFold  클래스 내 스플릿 함수를 통해 객체의 split()를 호출
# 폴드별로 학습용과 검증용 테스트의 로우 인덱스를 array로 반환한다
# 트레인 인덱스 내 5개의 어레이가 있는 것을 확인 할 수 있고, 그 어레이 내 값들이 로우 인덱스임
for train_index, test_index in kfold.split(features):
    print(train_index)
    print(test_index)
#인덱싱 번호를 주는거지 데이터 자체를 주는 게 아님

[ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 1

In [16]:
for train_index, test_index in kfold.split(features):
    # 반환된 인덱스를 이용하여 학습용과 검증용 테스트 데이터를 추출한다 (피처와 레이블 둘 다 편하고 동일하게 추출하기 위함) 
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    n_iter +=1
    #반복 시 마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기 : {2}, 검증 데이터의 크기: {3}'
         .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 :{1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도:',np.mean(cv_accuracy))
# 검증 정확도 값이 너무 낮다면 Overitting이거나 Underfitting 


#1 교차 검증 정확도 :1.0, 학습 데이터 크기 : 120, 검증 데이터의 크기: 30
#1 검증 세트 인덱스 :[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :1.0, 학습 데이터 크기 : 120, 검증 데이터의 크기: 30
#2 검증 세트 인덱스 :[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8333, 학습 데이터 크기 : 120, 검증 데이터의 크기: 30
#3 검증 세트 인덱스 :[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기 : 120, 검증 데이터의 크기: 30
#4 검증 세트 인덱스 :[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.8333, 학습 데이터 크기 : 120, 검증 데이터의 크기: 30
#5 검증 세트 인덱스 :[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.91998


In [18]:
import pandas as pd
iris = load_iris()
# 피쳐값을 데이터프레임으로 만들고, 칼럼명은 피처명으로
iris_df= pd.DataFrame(data = iris.data, columns = iris.feature_names)
# label이라는 새 컬럼을 넣어 해당 레이블 값 할당
iris_df['label'] = iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [30]:
kfold = KFold(n_splits = 3)
# 폴드 세트를 3번 반복할때마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호 반환
dt_clf = DecisionTreeClassifier(random_state=121)

n_iter = 0
for train_index, test_index in kfold.split(iris_df):
    n_iter +=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    
## 여기서 feature들을 3부분으로 나눴는데 학습에 1,2레이블 값이 50개씩, 검증에 0만 들어감
## 이는 단순 K Fold가 적절한 레이블 값의 분배가 되지 않아 검증이 제대로 되지 않음을 의미하

## 교차 검증 : 1
학습 레이블 데이터 분포:
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차 검증 : 2
학습 레이블 데이터 분포:
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차 검증 : 3
학습 레이블 데이터 분포:
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [37]:
##### kfold = KFold(n_splits = 3)
# 폴드 세트를 3번 반복할때마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호 반환
dt_clf = DecisionTreeClassifier(random_state=121)

n_iter = 0
for train_index, test_index in kfold.split(features):
    n_iter +=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
#추출한 인덱스에 맞는 학습 및 테스트 값을 할당

    print(f'{n_iter} : {label_train.value_counts()} \n')
    print(f'{n_iter} : {label_test.value_counts()}')
#학습 레이블에 해당하는 벨류 값의 개수를 추출
#테스트 레이블에 해당하는 벨류 값의 개수를 추출

    dt_clf.fit(X_train,y_train ) #학습시키고
    pred = dt_clf.predict(X_test) #학습을 바탕으로 테스트 피쳐로 예측 시킴
    print(pred) # 그 예측값을 출력 
    accuracy = np.round(accuracy_score(y_test,pred), 4) #그 예측값 출력
    print(accuracy)

1 : 1    50
2    50
Name: label, dtype: int64 

1 : 0    50
Name: label, dtype: int64
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
0.0
2 : 0    50
2    50
Name: label, dtype: int64 

2 : 1    50
Name: label, dtype: int64
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 0 2]
0.0
3 : 0    50
1    50
Name: label, dtype: int64 

3 : 2    50
Name: label, dtype: int64
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
0.0


In [27]:
from sklearn.model_selection import StratifiedKFold

#폴드 분배
skf = StratifiedKFold(n_splits = 3)
n_iter = 0

#features만 split할 때 필요할 뿐만 아니라 label값을 보면서 적절히 폴드에 분배
# 이 때 split 내 label을 꼭 삽입해야됨
for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter +=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('##교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

##교차 검증: 1
학습 레이블 데이터 분포:
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
1    17
2    16
Name: label, dtype: int64
##교차 검증: 2
학습 레이블 데이터 분포:
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
2    17
1    16
Name: label, dtype: int64
##교차 검증: 3
학습 레이블 데이터 분포:
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
2    17
0    16
Name: label, dtype: int64


In [38]:
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [35]:
kfold = KFold(n_splits = 3)
# 폴드 세트를 3번 반복할때마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호 반환
dt_clf = DecisionTreeClassifier(random_state=121)
val_list=[]
n_iter = 0
for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter +=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[train_index]
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    print(f'{n_iter} : {label_train.value_counts()} \n')
    print(f'{n_iter} : {label_test.value_counts()}')
    dt_clf.fit(X_train,y_train )
    pred = dt_clf.predict(X_test)
    print(pred)
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    print(accuracy)
    val_list.append(accuracy)
import numpy as np
val_acc = np.mean(val_list)
val_acc


1 : 2    34
0    33
1    33
Name: label, dtype: int64 

1 : 2    34
0    33
1    33
Name: label, dtype: int64
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2
 2 2 2 1 2 2 2 2 2 2 2 2 2]
0.98
2 : 1    34
0    33
2    33
Name: label, dtype: int64 

2 : 1    34
0    33
2    33
Name: label, dtype: int64
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 2 2 2 1
 2 2 2 2 2 2 2 2 2 2 2 2 2]
0.94
3 : 0    34
1    33
2    33
Name: label, dtype: int64 

3 : 0    34
1    33
2    33
Name: label, dtype: int64
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
0.98


0.9666666666666667

### KFold 클래스를 이용한 교차검증 방법
#### cross_val_score() 
k fold나 stratfiedfold를 한 줄로 요약한 것
cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)
폴드 세트를 추출하고, 학습/예측, 평가를 한번에 수행한다

첫 인자값이 estimator 예측위한 알고리즘(여기서는 DecisionTreeClassifier)
위치 매개변수로 없으면 절대로 안됨!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
verbose는 계산에 대한 설명을 해줌
X는 스프릿할 데이터, cv는 교차검증할 폴드 수(디폴트 값은 5)

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

#성능 지표는 정확도(accuracy), 교차 검증 세트 3개
#cross_val_score에 들어가는 하이퍼 파라미터는 적용할 알고리즘(esimator), 피쳐값, 레이블값, 평가지표, 폴드 수 
#사용한 알고리즘이 지도학습형 분류코드이기 때문에, 하이퍼 파라미터에 꼭 피쳐와 레이블 값 둘 다 들어가야 한다
#하이퍼 파라미터는 인자 순서 때문에 만약 디폴트로 두고 싶으면, 인자의 이름 = 인자값 으로 작성해야 한다
scores = cross_val_score(dt_clf, data, label, scoring = 'accuracy', cv = 3)
# 교차 검증을 3개 세트로 나누었기 때문에 scores 도 3개 데이터를 담은 리스트 값으로 나옴 
print('교차 검증별 정확도:',np.round(scores,4))
print('평균 검증 정확도 :',np.round(np.mean(scores),4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도 : 0.9667


### cross_validate()
cross_validate()는 다중 평가 지표로 모델을 평가할 때 여러가지 지표를 이용 가능
cross_validate(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', return_train_score=False, return_estimator=False, error_score=nan)
 cross_val_score와 비슷한 하이퍼 파라미터로 구성되지만, 분할마다 훈련과 테스트에 걸린 시간을 담은 딕셔너리를 반환
 fit_time(훈련시간)
 score_time(테스트시간)
 test_score(훈련점수)
 train_score(테스트점수)


In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import pandas as pd

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

scores = cross_validate(dt_clf, data, label, scoring = 'accuracy', cv = 3,return_train_score=True)

df = pd.DataFrame(scores, index =['case1','case2','case3'])
df

,fit_time,score_time,test_score,train_score
case1,0.000000,0.000000,0.98,1.0
case2,0.000000,0.000901,0.94,1.0
case3,0.000518,0.000000,0.98,1.0


## 파라미터가 너무 많다면?? -GridSearchCV
GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
최적의 하이퍼 파라미터를 찾는 과정

가능한 파라미터들을 나열해놓고, 그리드 형식으로 서치하는 것

cv=none이기 때문에 굳이 폴드를 나눠서 교차검증을 안할 수도 있음

예를들어 파라미터가 2개이며 각각 3개의 후보가 있다면 총 그리드는 9개 

이 9개 그리드 중 accuracy 높은 최적의 조합인 하이퍼 파라미터 값을 찾는 것이 GridSearchCV



In [58]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]} #파라미터 후보들을 할당 (딕셔너리 형태로 나옴)
svc = svm.SVC() #svm 알고리즘 불러와서 선언
clf = GridSearchCV(svc, parameters) #그리드 서치의 estimator 알고리즘은 svc고, 파라미터는 선언한 변수로 대체
clf.fit(iris.data, iris.target)


GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

In [81]:
# grid_tree.best_params_ 프린트 해보기

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd

#데이터를 로딩하고 학습, 테스트 데이터로 분리
iris = load_iris()
X_train,X_test,y_train,y_test = train_test_split(iris_data.data, iris_data.target,
                                                test_size=0.2,random_state=121)

dtree = DecisionTreeClassifier()

### 파라미터들을 dictionary형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

#param_grid의 하이퍼 파라미터들을 3개의 train,test set fold로 나누어서 테스트 수행 설정
### refit=True가 default로 가장 좋은 파라미터를 이용해 재학습 시킴
grid_dtree = GridSearchCV(dtree, param_grid = parameters, cv = 3, refit = True)

# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습하고 평가
grid_dtree.fit(X_train, y_train)

#GridSearchCV 결과를 추출하여 데이터프레임으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params','mean_test_score','rank_test_score',\
          'split0_test_score','split1_test_score','split2_test_score']]




,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000353,0.000500,0.000229,0.000324,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.700,0.7,0.70,0.700000,1.110223e-16,5
1,0.000397,0.000434,0.000000,0.000000,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.700,0.7,0.70,0.700000,1.110223e-16,5
2,0.000696,0.000494,0.000000,0.000000,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.925,1.0,0.95,0.958333,3.118048e-02,3
3,0.000700,0.000497,0.000337,0.000476,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.925,1.0,0.95,0.958333,3.118048e-02,3
4,0.000332,0.000469,0.000328,0.000464,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.975,1.0,0.95,0.975000,2.041241e-02,1


In [77]:
print(grid_dtree.best_params_)

{'max_depth': 3, 'min_samples_split': 2}


In [87]:
X=[[5.1, 3.5, 1.4, 0.3]]
grid_dtree.predict(X) #첫 어레이에 해당하는 세토사? 그 꽃으로 예측

array([0])

### 과적합을 방지하기 위한 방법 중 하나는 학습시 검증방법인 KFold
### 레이블 값의 공평분배를 위한 Stratified KFold
### for문으로 굴리지 않고 한 줄로 처리하는 cross_val_score()
### cross_val_score()의 가장 적절한 하이퍼 파라미터를 찾는 GridSearchCv